In [1]:
from hsmm_core.data_utils import TradingHours, DataLoader
from hsmm_core.data_utils import get_files_for_ticker
from hsmm_core.labelling import DataLabellingSimple
from hsmm_core.consts import ThresholdMethod, LabellingChoice
import datetime as dt
import os
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
plt.style.use(os.path.join(mpl.get_configdir(), 'latexstyle.mplstyle'))
# for debugging
import pdb
from IPython.core.debugger import set_trace

In [2]:
def describe_helper(series):
    splits = str(series.describe()).split()
    keys, values = "", ""
    for i in range(0, len(splits), 2):
        keys += "{:8}\n".format(splits[i])
        values += "{:>8}\n".format(splits[i+1])
    return keys, values

In [3]:
def get_symbol_path(symbolIdx):
    symbol_path = os.path.join(nonDirectional, sorted(symbols)[symbolIdx], 'NON_DIRECTIONAL')
    return symbol_path

def list_of_symbol_dates(symbolIdx):
        return [f.split(".")[0] for f in next(os.walk(get_symbol_path(symbolIdx)))[2]]


## Labels

In [4]:
#nonDirectional contains ONLY the label files for each of the symbols. each of the labels files has the correct number of labels

nonDirectional = os.path.join('/media/ak/DataOnly/ProcessedData/labels/') 
dataOnlyDrive = os.path.join('/media/ak/DataOnly/FinDataReal/')

# symbols and their NON _DIRECTIONAL LABEL FILES
symbols = sorted(next(os.walk(nonDirectional))[1])


In [5]:
number_of_symbols = len(symbols)
print(number_of_symbols)

# each containing a lot of the symbols that I need with the rest of the data


51


In [6]:
trading_hours_filter = TradingHours.only_mkt_hours

# data_loader = DataLoader(trading_hours_filter, ccy_bar=1000)

# data = data_loader.load_trades_data(ticker, start_date=sd, end_date=ed)

labelling_method_params = [
{
    'labelling_method': LabellingChoice.ma_in_window,
    'rolling_window': 10,
    # Uncomment below if you want to check a price move only above a certain level
    'updown_threshold': 0.0001,
    'threshold_method': ThresholdMethod.arbitrary,},
{
    'labelling_method': LabellingChoice.ma_in_window,
    'rolling_window': 8,
    # Uncomment below if you want to check a price move only above a certain level
    'updown_threshold': 0.0002,
    'threshold_method': ThresholdMethod.arbitrary,},
{
    'labelling_method': LabellingChoice.price_move_in_window,
    'rolling_window': 24,
    # Uncomment below if you want to check a price move only above a certain level
    'updown_threshold': 0.005,
    'threshold_method': ThresholdMethod.arbitrary,}
    ,
{
    'labelling_method': LabellingChoice.ep_in_window,
    'rolling_window': 9,
    # Uncomment below if you want to check a price move only above a certain level
    'updown_threshold': 0.0008,
    'threshold_method': ThresholdMethod.arbitrary,},
    {
    'labelling_method': LabellingChoice.ma_in_window,
    'rolling_window': 5,
    # Uncomment below if you want to check a price move only above a certain level
    'updown_threshold': 0.018},
    {
    'labelling_method': LabellingChoice.ma_in_window,
    'rolling_window': 15,
    # Uncomment below if you want to check a price move only above a certain level
    'updown_threshold': 0.0018}]



In [7]:
alternate_label_values =['label_PrMov__window_5__thres_arbitrary__0.1',  
                         'label_MovAv__window_10__thres_arbitrary__0.0',
                         'label_MovAv__window_8__thres_arbitrary__0.0',  
                         'label_PrMov__window_24__thres_arbitrary__0.5',
                         'label_EndPo__window_9__thres_arbitrary__0.1',  
                         'label_MovAv__window_5__thres_arbitrary__1.8',  
                         'label_MovAv__window_15__thres_arbitrary__0.2']

LabelsAlternateNames = ['LabelsAlternateOne', 'LabelsAlternateTwo',  
                        'LabelsAlternateThree', 'LabelsAlternateFour',  
                        'LabelsAlternateFive', 'LabelsAlternateSix', 
                        'LabelsAlternateSeven']


In [41]:

### 
index =19 # correspond to a symbol
# for index in range(20,56):
print("index --->", index)
ticker = str(symbols[index])
print(ticker)
nonDirectionalSymbol = get_symbol_path(index) # the non-directional labels path
print(nonDirectionalSymbol)
listofDates = sorted(get_files_for_ticker(ticker))
print ('Ticker', ticker, ' and labels', len(listofDates))
# v stupid check but necessary- maybe cache is the issue but whatever
print(sorted(listofDates)[len(listofDates)-1] ==listofDates[-1])

## step 1
sd = listofDates[1] # this must be set to 0 otherwirse it is problematic
ed = listofDates[2] # added a check before
print(sd, ed)

index ---> 19
IEER.L
/media/ak/DataOnly/ProcessedData/labels/IEER.L/NON_DIRECTIONAL
Ticker IEER.L  and labels 110
True
20170704 20170705


In [48]:

# trading_hours_filter = TradingHours.only_mkt_hours

# data_loader = DataLoader(trading_hours_filter)

# data = data_loader.load_trades_data(ticker, start_date=sd, end_date=ed)

## Label construction starts here

In [51]:

### 
#index =26 # correspond to a symbol
# for index in [39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56]:
    print("index --->", index)
    ticker = str(symbols[index])
    print(ticker)
    nonDirectionalSymbol = get_symbol_path(index) # the non-directional labels path
    print(nonDirectionalSymbol)
    listofDates = sorted(get_files_for_ticker(ticker))
    print ('Ticker', ticker, ' and labels', len(listofDates))
    # v stupid check but necessary- maybe cache is the issue but whatever
    print(sorted(listofDates)[len(listofDates)-1] ==listofDates[-1])

    ## step 1
    sd = listofDates[0] # this must be set to 0 otherwirse it is problematic
    ed = listofDates[len(listofDates)-1] # added a check before

    trading_hours_filter = TradingHours.only_mkt_hours

    data_loader = DataLoader(trading_hours_filter)

    data = data_loader.load_trades_data(ticker, start_date=sd, end_date=ed)
    # this is the core of label creation ## 
    for label_init in labelling_method_params:
        labeller = DataLabellingSimple(label_init)
        the_label = labeller.get_label_name()
        labeller.label_training_data(data)
    print('Finished Creating Labels')  
    # print((list(data.keys())))

    ## step 2 - this is BS
    print(data[list(data.keys())[0]].filter(like='label', axis=1).columns.values)
    ## step 3
    for idex , dateIdex in enumerate(list(data.keys())):
        print(idex)
        print('Date:', dateIdex)
        for alternate_label_value_idex, alternate_label_value in enumerate(alternate_label_values):
            print('Current Alternate Label Value: ',LabelsAlternateNames[alternate_label_value_idex])        
            df = data[dateIdex][['TradedTime', 'TradedPrice', 'ReturnTradedPrice', 'Volume',
           'Duration',str(alternate_label_value)]]

            label_symbol_date_path = os.path.join(dataOnlyDrive,LabelsAlternateNames[alternate_label_value_idex], ticker)

            filename = os.path.join(label_symbol_date_path,dateIdex+'.csv')

            if os.path.exists(label_symbol_date_path) ==True:
                print('Saving the dataframce CSV')
                print('path_exists')
                df.to_csv(filename)
            else:
                os.makedirs(label_symbol_date_path)
                print('Produced this directory: ', label_symbol_date_path)
                df.to_csv(filename)
        print('### Finished###')

index ---> 39
RTO.L
/media/ak/DataOnly/ProcessedData/labels/RTO.L/NON_DIRECTIONAL
Ticker RTO.L  and labels 78
True
Finished Creating Labels
['label_PrMov__window_5__thres_arbitrary__0.1'
 'label_MovAv__window_10__thres_arbitrary__0.0'
 'label_MovAv__window_8__thres_arbitrary__0.0'
 'label_PrMov__window_24__thres_arbitrary__0.5'
 'label_EndPo__window_9__thres_arbitrary__0.1'
 'label_MovAv__window_5__thres_arbitrary__1.8'
 'label_MovAv__window_15__thres_arbitrary__0.2']
0
Date: 20170703
Current Alternate Label Value:  LabelsAlternateOne
Produced this directory:  /media/ak/DataOnly/FinDataReal/LabelsAlternateOne/RTO.L
Current Alternate Label Value:  LabelsAlternateTwo
Produced this directory:  /media/ak/DataOnly/FinDataReal/LabelsAlternateTwo/RTO.L
Current Alternate Label Value:  LabelsAlternateThree
Produced this directory:  /media/ak/DataOnly/FinDataReal/LabelsAlternateThree/RTO.L
Current Alternate Label Value:  LabelsAlternateFour
Produced this directory:  /media/ak/DataOnly/FinDataRea

Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
### Finished###
13
Date: 20170720
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
### Finished###
14
Date: 20170721
Current Alternate Label Value:  L

### Finished###
29
Date: 20170811
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
### Finished###
30
Date: 20170814
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  La

Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
### Finished###
43
Date: 20170901
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the data

Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
### Finished###
58
Date: 20170922
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value

...

Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
### Finished###
70
Date: 20170908
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
### Finished###
71
Date: 20170911
Current Alternate Label Value:  L

Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
### Finished###
84
Date: 20170928
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
### Finished###
85
Date: 20170929
Current Alternate Label Value:  L

Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
### Finished###
97
Date: 20180316
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
### Finished###
98
Date: 20180319
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  La

Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
### Finished###
111
Date: 20180409
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Valu

IndexError: list index out of range

In [17]:
# for idex , dateIdex in enumerate(list(data.keys())):
#     print(idex)
#     print('Date:', dateIdex)
#     for alternate_label_value_idex, alternate_label_value in enumerate(alternate_label_values):
#         print('Current Alternate Label Value: ',LabelsAlternateNames[alternate_label_value_idex])        
#         df = data[dateIdex][['TradedTime', 'TradedPrice', 'ReturnTradedPrice', 'Volume',
#        'Duration',str(alternate_label_value)]]

#         label_symbol_date_path = os.path.join(dataOnlyDrive,LabelsAlternateNames[alternate_label_value_idex], ticker)

#         filename = os.path.join(label_symbol_date_path,dateIdex+'.csv')

#         if os.path.exists(label_symbol_date_path) ==True:
#             print('Saving the dataframce CSV')
#             print('path_exists')
#             df.to_csv(filename)
#         else:
#             os.makedirs(label_symbol_date_path)
#             print('Produced this directory: ', label_symbol_date_path)
#             df.to_csv(filename)


0
Date: 20170116
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
1
Date: 20170117
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataf

Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
17
Date: 20170710
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_

Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
34
Date: 20170802
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
35
Date: 20170803
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the da

Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
51
Date: 20170825
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
52
Date: 20170829
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the data

Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFive
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSix
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateSeven
Saving the dataframce CSV
path_exists
67
Date: 20170919
Current Alternate Label Value:  LabelsAlternateOne
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateTwo
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateThree
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlternateFour
Saving the dataframce CSV
path_exists
Current Alternate Label Value:  LabelsAlterna

## example visualisation to be moved to a different script


In [ ]:
from IPython.display import display
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
from statsmodels.formula.api import ols
import seaborn as sns
import string

In [ ]:
keys=list(data.keys())
from collections import defaultdict
labelsDict = defaultdict(dict)

for key in keys:
    df =pd.DataFrame(data[key])
    differentLabelTypes = df.filter(like='label', axis=1).columns.values
    print(differentLabelTypes)
    print(differentLabelTypes.shape[0])
    for i in range(differentLabelTypes.shape[0]):
        #print(i)
        try:
            #print(key)
            labelsDict[differentLabelTypes[i]][key] = df[differentLabelTypes[i]].value_counts()[1]/df[differentLabelTypes[i]].value_counts()[0]
        except:
            print('continuing')
            continue

In [ ]:
dfFromDict = df.from_dict(labelsDict)
dfDesc = df.from_dict(labelsDict).describe()


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
balanceKeys = labelsDict.keys()

df.from_dict(labelsDict)[differentLabelTypes[1]].hist(bins=30)
plt.axvline(df.from_dict(labelsDict)[differentLabelTypes[1]].median(), color='r', linestyle='dashed', linewidth=1)
x= df.from_dict(labelsDict)[differentLabelTypes[1]]
print([differentLabelTypes[3]])
min_ylim, max_ylim = plt.ylim()
# plt.figtext(x.median()*0.3, max_ylim*0.9, 'Median: {:.2f}'.format(x.median()), fontsize =15)
# plt.figtext(x.std()*1.1, max_ylim*0.7, 'StDev: {:.2f}'.format(x.std()), fontsize =15)
# plt.annotate(r"$\}$",fontsize=24,
#             xy=(0.27, 0.77), xycoords='figure fraction'
#             )
# plt.text(13.1,1, text , fontsize=15)
plt.show()

In [ ]:
df.from_dict(labelsDict)[differentLabelTypes[1]].hist(bins=20)
plt.axvline(df.from_dict(labelsDict)[differentLabelTypes[1]].median(), color='r', linestyle='dashed', linewidth=1)
x= df.from_dict(labelsDict)[differentLabelTypes[0]]
print([differentLabelTypes[3]])
min_ylim, max_ylim = plt.ylim()
plt.text(2.3, 6.9, 'Median: {:.2f}'.format(x.median()), fontsize =15)
# plt.text(x.std()*1.1, max_ylim*0.7, 'StDev: {:.2f}'.format(x.std()), fontsize =15)
# plt.annotate(r"$\}$",fontsize=24,
#             xy=(0.27, 0.77), xycoords='figure fraction'
#             )
# plt.figtext(.95, .49, describe_helper(pd.Series(x))[0], {'multialignment':'left'})
# plt.figtext(1.05, .49, describe_helper(pd.Series(x))[1], {'multialignment':'right'})
plt.show()




In [ ]:
import seaborn as sns
from scipy.stats import norm
import numpy as np
def price_based_histogram_labels(x, labelText):
    x= np.array(x.dropna())
    ax = sns.distplot(x, fit=norm, kde=True, bins=20)
   # _=ax.text(x.std()*0.5, max_ylim*0.5, 'Median: {:.2f}'.format(np.median(x)), {'multialignment':'right'})
    _=ax.axvline(np.median(x), color='r', linestyle='dashed', linewidth=2)
    #_=ax.axhline(np.std(x)*0.9, color='r', linestyle='dashed', linewidth=1)
    _=ax.text(0.9, 1.1, 'StDev: {:.2f}'.format(x.std()))
    #plt.title('Rolling window of ' + labelText.split("_")[4] + ' , threshold ' + labelText.split("_")[-1] )
    plt.tight_layout() 
    plt.show()
for i in range(0,5):
    x= df.from_dict(labelsDict)[differentLabelTypes[i]]
    price_based_histogram_labels(x, labelText= differentLabelTypes[i])

In [ ]:
fig, ax = plt.subplots()
labelIdx =0
label = str(differentLabelTypes[labelIdx])
x= np.array(df.from_dict(labelsDict)[differentLabelTypes[labelIdx]].dropna())
ax = sns.distplot(x, fit=norm, kde=True, bins=30, label =label)
_=ax.axvline(np.median(x), color='r', linestyle='dashed', linewidth=2)
ax.set_xlabel('Proportion of labels') # <--------------- Set x axis label
ax.set_ylabel('Frequency') # <--------- Set y axis label
ax.legend(loc='best') # <-------- Move the legend
sns.despine()